# FitsFileGroup and Database

Currently astropop uses simple [Astropy Table](https://docs.astropy.org/en/stable/table/) to store headers data in `FitsFileGroup`. This can be fast, but have the problem to be in-memory only.

So, we will investigate the use of SQL database to store the header table and filtering.

Fortunately, Python has a built-in sqlite module called [sqlite3](https://docs.python.org/3/library/sqlite3.html), that we can use here.

## SQLite3 Wrapper

First of all, lest implement a wrapper to do a more object-oriented sql work. This is limited to a single table database and just some functions and interactions.

In [1]:
# file _db.py
import sqlite3 as sql
import numpy as np

from astropop.logger import logger


np_to_sql = {
    'i': 'INTEGER',
    'f': 'REAL',
    'S': 'TEXT',
    'U': 'TEXT',
    'b': 'BOOLEAN',
}


def _sanitize_colnames(colnames):
    """Sanitize the colnames to avoid invalid characteres like '-'."""
    return [c.replace('-', '_').replace(' ', '_').lower() for c in colnames]


class Database:
    """Database creation and manipulation with SQL."""

    def __init__(self, db, table='table', dtype=None):
        self._db = db
        self._con = sql.connect(self._db)
        self._cur = self._con.cursor()
        self._table = table
        self._add_table(dtype)

    def execute(self, command):
        """Execute a SQL command in the database."""
        logger.debug('executing sql command: "%s"',
                     str.replace(command, '\n', ' '))
        self._cur.execute(command)
        res = self._cur.fetchall()
        self._con.commit()
        return res

    def _add_table(self, dtype):
        """Create a table in database."""
        logger.debug('Initializing "%s" table.', self._table)
        tables = [i[0] for i in self.execute("SELECT name FROM sqlite_master "
                                             "WHERE type='table';")]
        if self._table in tables:
            logger.debug('table "%s" already exists', self._table)
            return
        comm = f"CREATE TABLE {self._table}"
        comm += " (\n_id INTEGER PRIMARY KEY AUTOINCREMENT"
        if dtype is not None:
            comm += ",\n"
            for i, name in enumerate(dtype.names):
                kind = dtype[i].kind
                comm += f"\t'{name}' {np_to_sql[kind]}"
                if i != len(dtype) - 1:
                    comm += ",\n"
        comm += "\n);"
        self.execute(comm)

    def colnames(self):
        """Get the column names of the current table."""
        self.execute(f"SELECT * FROM {self._table} WHERE 1=0")
        return [i[0].lower() for i in self._cur.description]

    def add_column(self, column, dtype):
        """Add a column to a table."""
        col = _sanitize_colnames([column])[0]
        comm = f"ALTER TABLE {self._table} ADD COLUMN '{col}' "
        logger.debug('adding column "%s" "%s" "%s"', col, dtype, dtype.kind)
        comm += f"{np_to_sql[dtype.kind]};"
        self.execute(comm)

    def add_row(self, data, add_columns=False):
        """Add a dict row to a table.

        Parameters
        ----------
        data : dict
            Dictionary of data to add.
        add_columns : bool (optional)
            If True, add missing columns to the table.
        """
        data_c = {}
        sanitized = _sanitize_colnames(data.keys())
        # create a dict copy with sanitized keys
        for k, ks in zip(data.keys(), sanitized):
            data_c[ks] = data[k]

        if add_columns:
            # add missing columns
            cols = set(self.colnames())
            for k in data_c.keys():
                if k not in cols:
                    self.add_column(k, np.array([data_c[k]]).dtype)

        # create the sql command and add the row
        cols = self.colnames()
        comm = f"INSERT INTO {self._table} VALUES ("
        for i, name in enumerate(cols):
            if name in data_c.keys():
                d = data_c[name]
                if isinstance(d, str):
                    d = f"'{d}'"
                comm += f"{d}"
            elif name == '_id':
                comm += "NULL"
            else:
                comm += "NULL"
            if i != len(cols) - 1:
                comm += ", "
        comm += ");"
        self.execute(comm)

    def select(self, columns=None, where=None):
        """Select rows from a table.

        Parameters
        ----------
        columns : list (optional)
            List of columns to select. If None, select all columns.
        where : dict (optional)
            Dictionary of conditions to select rows. Keys are column names,
            values are values to compare. All rows equal to the values will
            be selected. If None, all rows are selected.
        """
        if columns is None:
            columns = '*'
        else:
            # only use sanitized column names
            columns = ', '.join(_sanitize_colnames(columns))

        if where is None:
            _where = '1=1'
        else:
            for i, (k, v) in enumerate(where.items()):
                if isinstance(v, str):
                    # avoid sql errors
                    v = f"'{v}'"
                if i == 0:
                    _where = f"{k}={v}"
                else:
                    _where += f" AND {k}={v}"
        return self.execute(f"SELECT {columns} FROM {self._table} "
                            f"WHERE {_where}")

    def __len__(self):
        """Get the number of rows in the current table."""
        return self.execute(f"SELECT COUNT(*) FROM {self._table}")[0][0]

    def __del__(self):
        """Delete the class, closing the db connection."""
        # ensure connection is closed.
        self._con.close()

In [2]:
from astropy.io import fits
from astropop.file_collection import list_fits_files
from astropop.logger import logger

logger.setLevel('INFO')

In [3]:
files = list_fits_files('/home/julio/19jan30', fits_extensions=['fits.gz'])
logger.info('number of files %i', len(files))

2022-04-07 23:21:46,350 astropop - INFO - number of files 427  [707346715]


Is everything working?

In [4]:
# create a in-memory database
db = Database(':memory:', 'headers')

In [5]:
def add_files(files):
    for i, f in enumerate(files):
        logger.debug("file %i from %i", i+1, len(files))
        header = dict(fits.getheader(f))
        header.pop('COMMENT', None)
        header.pop('HISTORY', None)
        db.add_row(header, add_columns=True)

In [6]:
add_files(files)

In [7]:
db.select(columns=['image', 'object', 'filter', 'lamina', 'bitpix', 'readtime'], where={'OBJECT': 'HD126593'})

[('HD126593_STD_L0_F2', 'HD126593', 'F2', 'L0', 16, 1e-06),
 ('HD126593_STD_L1_F2', 'HD126593', 'F2', 'L1', 16, 1e-06),
 ('HD126593_STD_L2_F2', 'HD126593', 'F2', 'L2', 16, 1e-06),
 ('HD126593_STD_L3_F2', 'HD126593', 'F2', 'L3', 16, 1e-06),
 ('HD126593_STD_L4_F2', 'HD126593', 'F2', 'L4', 16, 1e-06),
 ('HD126593_STD_L5_F2', 'HD126593', 'F2', 'L5', 16, 1e-06),
 ('HD126593_STD_L6_F2', 'HD126593', 'F2', 'L6', 16, 1e-06),
 ('HD126593_STD_L7_F2', 'HD126593', 'F2', 'L7', 16, 1e-06),
 ('HD126593_STD_L8_F2', 'HD126593', 'F2', 'L8', 16, 1e-06),
 ('HD126593_STD_L9_F2', 'HD126593', 'F2', 'L9', 16, 1e-06),
 ('HD126593_STD_LA_F2', 'HD126593', 'F2', 'LA', 16, 1e-06),
 ('HD126593_STD_LB_F2', 'HD126593', 'F2', 'LB', 16, 1e-06),
 ('HD126593_STD_LC_F2', 'HD126593', 'F2', 'LC', 16, 1e-06),
 ('HD126593_STD_LD_F2', 'HD126593', 'F2', 'LD', 16, 1e-06),
 ('HD126593_STD_LE_F2', 'HD126593', 'F2', 'LE', 16, 1e-06),
 ('HD126593_STD_LF_F2', 'HD126593', 'F2', 'LF', 16, 1e-06)]

In [8]:
print('columns: ', db.colnames())
print('files:', len(db))

columns:  ['_id', 'simple', 'bitpix', 'naxis', 'naxis1', 'naxis2', 'extend', 'bzero', 'bscale', 'head', 'acqmode', 'readmode', 'imgrect', 'hbin', 'vbin', 'subrect', 'datatype', 'xtype', 'xunit', 'raywave', 'calbwvnm', 'trigger', 'calib', 'dllver', 'exposure', 'temp', 'readtime', 'operatn', 'gain', 'emrealgn', 'vclkamp', 'vshift', 'outptamp', 'preamp', 'serno', 'unsttemp', 'blclamp', 'precan', 'flipx', 'flipy', 'cntcvtmd', 'cntcvt', 'dtnwlgth', 'sntvty', 'spsnfltr', 'thrshld', 'pcntenld', 'nsethsld', 'ptnthld1', 'ptnthld2', 'ptnthld3', 'ptnthld4', 'avgftrmd', 'avgfctr', 'frmcnt', 'portmode', 'lsheight', 'lsspeed', 'lsaltdir', 'lsctrl', 'lsdir', 'fksmode', 'fktmode', 'usertxt1', 'usertxt2', 'usertxt3', 'usertxt4', 'date', 'frame', 'eshtmode', 'preampgaintext', 'spectrographserial', 'shamrockisactive', 'spectrographname', 'spectrographisactive', 'irigdataavailable', 'irigdata', 'detector', 'exptime', 'camgain', 'date_obs', 'ra', 'dec', 'epoch', 'airmass', 'jd', 'st', 'ha', 'image', 'teles

It looks like everything is working properly. All 427 files were added and all columns are present. What is int seems to be returned as int and what is float seems to be float. The filtering is also working too.

Notes:

- SQL tables don't like `-` character in name. So it is replaced by `_`. Same for spaces. This create possible conflicts, but may be a smaller problem in practice.

# FitsFileGroup implementations

Lets try our two `FitsFileGroup` implementations. One using `astropy.table.Table`, another using our SQL wrapper.

## Table implementation

In [11]:
from astropy.table import Table
from astropop.file_collection import list_fits_files

from astropop.fits_utils import _fits_extensions, \
                                _fits_extensions_with_compress
from astropop.framedata import check_framedata
from astropop.py_utils import check_iterable
from astropop.logger import logger


def create_table_summary(headers, n):
    """Create a table summary of headers.

    Parameters
    ----------
    headers: iterator
        Iterator for a list of header files.
    n: int
        Number of headers to iterate.
    """
    summary_dict = {}
    for i, head in enumerate(headers):
        logger.debug('Reading file %i from %i', i, n)
        keys = head.keys()
        for k in keys:
            k_lower = k.lower()
            if k_lower in ('history', 'comment'):
                logger.debug('%s key ignored', k)
                continue
            if k_lower not in summary_dict.keys():
                summary_dict[k_lower] = [None]*n
            summary_dict[k_lower][i] = head.get(k)

    return Table(summary_dict)


def gen_mask(table, keywords):
    """Generate a mask to be applyed in the filtering."""
    if len(table) == 0:
        return []

    t = Table(table)

    mask = np.ones(len(t), dtype=bool)
    for k, v in keywords.items():
        if not check_iterable(v):
            v = [v]
        k = k.lower()
        if k not in t.colnames:
            t[k] = [None]*len(t)
        nmask = [t[k][i] in v for i in range(len(t))]
        mask &= np.array(nmask)

    return mask


class FitsFileGroup_Table():
    """Easy handle groups of fits files."""

    def __init__(self, location=None, files=None, ext=0,
                 compression=False, **kwargs):
        self._ext = ext
        self._extensions = kwargs.get('fits_ext',
                                      _fits_extensions_with_compress
                                      if compression else _fits_extensions)

        self._include = kwargs.get('glob_include')
        self._exclude = kwargs.get('glob_exclude')
        self._keywords = kwargs.get('keywords')

        if location is None and files is None:
            raise ValueError("You must specify a 'location'"
                             "or a list of 'files'")
        if files is None and location is not None:
            files = list_fits_files(location, self._extensions,
                                    self._include, self._exclude)

        self._files = files
        self._location = location

        self._summary = create_table_summary(self.headers(), len(self))

    def __len__(self):
        return len(self.files)

    @property
    def files(self):
        return self._files.copy()

    @property
    def location(self):
        return self._location

    @property
    def keywords(self):
        return self._keywords

    @property
    def summary(self):
        return Table(self._summary)

    def __copy__(self, files=None, summary=None):
        nfg = FitsFileGroup_Table.__new__(FitsFileGroup_Table)
        for k, v in self.__dict__.items():
            if k == '_summary':
                nfg._summary = summary or self._summary
            elif k == '_files':
                nfg._files = files if files is not None else self._files
            else:
                nfg.__dict__[k] = v
        return nfg

    def __getitem__(self, item):
        if isinstance(item, str):
            # string will be interpreted as collumn name
            if item.lower() not in self._summary.colnames:
                raise KeyError(f'Column {item} not found.')
            return self._summary.columns[item.lower()]

        # returning FitsFileGroups
        if isinstance(item, (int, np.integer)):
            # single index will be interpreted as a single file group
            return self.__copy__(files=[self._files[item]],
                                 summary=self._summary[item])
        if (isinstance(item, slice)):
            files = self._files[item]
            summ = self._summary[item]
            return self.__copy__(files=files, summary=summ)
        if isinstance(item, (np.ndarray, list, tuple)):
            item = np.array(item)
            if len(item) == 0:
                return self.__copy__(files=[], summary=self._summary[item])
            files = list(np.take(self._files, item))
            summ = self._summary[item]
            return self.__copy__(files=files, summary=summ)

        raise KeyError(f'{item}')

    def filtered(self, keywords=None):
        """Create a new FileGroup with only filtered files."""
        where = np.where(gen_mask(self._summary, keywords))[0]
        return self[where]

    def values(self, keyword, unique=False):
        """Return the values of a keyword in the summary.

        If unique, only unique values returned.
        """
        if keyword not in self.summary.colnames:
            if unique:
                n = 1
            else:
                n = len(self.summary)
            return [None]*n
        if unique:
            return list(set(self.summary[keyword].tolist()))
        return self.summary[keyword].tolist()

    def add_column(self, name, values, mask=None):
        """Add a new column to the summary."""
        if not check_iterable(values):
            values = [values]*len(self.summary)
        elif len(values) != len(self.summary):
            values = [values]*len(self.summary)

        self.summary[name] = values
        self.summary[name].mask = mask

    def _intern_yelder(self, files=None, ext=None, ret_type=None,
                       **kwargs):
        """Iter over files."""
        ext = ext if ext is not None else self._ext
        files = files if files is not None else self._files
        for i in files:
            if ret_type == 'header':
                yield fits.open(i, **kwargs)[ext].header
            if ret_type == 'data':
                yield fits.open(i, **kwargs)[ext].data
            if ret_type == 'hdu':
                yield fits.open(i, **kwargs)[ext]
            if ret_type == 'framedata':
                yield check_framedata(i, hdu=ext, **kwargs)

    def hdus(self, ext=None, **kwargs):
        """Read the files and iterate over their HDUs."""
        return self._intern_yelder(ext=ext, ret_type='hdu', **kwargs)

    def headers(self, ext=None, **kwargs):
        """Read the files and iterate over their headers."""
        return self._intern_yelder(ext=ext, ret_type='header', **kwargs)

    def data(self, ext=None, **kwargs):
        """Read the files and iterate over their data."""
        return self._intern_yelder(ext=ext, ret_type='data', **kwargs)

    def framedata(self, ext=None, **kwargs):
        """Read the files and iterate over their data."""
        return self._intern_yelder(ext=ext, ret_type='framedata', **kwargs)

In [12]:
f = FitsFileGroup_Table(files=files)
f.filtered({'object': 'HD126593'}).summary

simple,bitpix,naxis,naxis1,naxis2,extend,bzero,bscale,head,acqmode,readmode,imgrect,hbin,vbin,subrect,datatype,xtype,xunit,raywave,calbwvnm,trigger,calib,dllver,exposure,temp,readtime,operatn,gain,emrealgn,vclkamp,vshift,outptamp,preamp,serno,unsttemp,blclamp,precan,flipx,flipy,cntcvtmd,cntcvt,dtnwlgth,sntvty,spsnfltr,thrshld,pcntenld,nsethsld,ptnthld1,ptnthld2,ptnthld3,ptnthld4,avgftrmd,avgfctr,frmcnt,portmode,lsheight,lsspeed,lsaltdir,lsctrl,lsdir,fksmode,fktmode,usertxt1,usertxt2,usertxt3,usertxt4,date,frame,eshtmode,preampgaintext,spectrographserial,shamrockisactive,spectrographname,spectrographisactive,irigdataavailable,irigdata,detector,exptime,camgain,date-obs,ra,dec,epoch,airmass,jd,st,ha,image,telescop,platescl,instrume,object,observer,rdnoise,lamina,lam-pos,filter,fil-pos,focusval,w-bar,w-temp,w-hum
bool,int64,int64,int64,int64,bool,int64,int64,str1,str6,str5,str15,str1,str1,str14,str6,str12,int64,float64,int64,str8,str11,str13,str9,str5,float64,int64,str3,int64,int64,str8,str12,str4,str4,str4,bool,int64,int64,int64,int64,int64,float64,float64,int64,float64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int64,str1,str1,str1,str1,str19,str23,int64,str1,str1,int64,str1,int64,int64,str1,str1,str9,str4,str23,str8,str9,str6,str5,str13,str8,str8,str18,str9,str4,str10,str12,str28,str3,str2,str2,str2,str1,str4,str5,str4,str2
True,16,2,1024,1024,True,32768,1,,Single,Image,"1, 1024,1024, 1",1,1,"1, 1024,1024,1",Counts,Pixel number,0,0.0,0,Internal,"0.,1.,0.,0.",2.100.30039.0,"5,00000","-74,1",1e-06,4,1.5,0,0,12.9E-06,Conventional,2.5x,4335,-999,False,0,0,0,0,0,500.0,0.0,0,0.0,0,0,0.0,0.0,0.0,2.802597e-45,0,1,1,-1,0,0.0,0,0,0,0,0,,,,,2019-01-31T06:44:29,2019-01-31T06:44:29.186,0,,,0,,0,0,,,"5,00000",2.5x,2019-01-31T06:44:29.186,14:28:50,-60:32:25,2000.0,1.390,2458514.78087,12:22:55,-2:07:16,HD126593_STD_L0_F2,0.60m(BC),0.34,Ikon 19002,HD126593,Julio Cesar Neves Campagnolo,6.3,L0,0,F2,2,2247,"756,0","18,8",51
True,16,2,1024,1024,True,32768,1,,Single,Image,"1, 1024,1024, 1",1,1,"1, 1024,1024,1",Counts,Pixel number,0,0.0,0,Internal,"0.,1.,0.,0.",2.100.30039.0,"5,00000","-74,1",1e-06,4,1.5,0,0,12.9E-06,Conventional,2.5x,4335,-999,False,0,0,0,0,0,500.0,0.0,0,0.0,0,0,0.0,0.0,0.0,2.802597e-45,0,1,1,-1,0,0.0,0,0,0,0,0,,,,,2019-01-31T06:44:41,2019-01-31T06:44:41.496,0,,,0,,0,0,,,"5,00000",2.5x,2019-01-31T06:44:41.496,14:28:50,-60:32:25,2000.0,1.389,2458514.78101,12:23:07,-2:07:04,HD126593_STD_L1_F2,0.60m(BC),0.34,Ikon 19002,HD126593,Julio Cesar Neves Campagnolo,6.3,L1,1,F2,2,2247,"756,0","18,8",51
True,16,2,1024,1024,True,32768,1,,Single,Image,"1, 1024,1024, 1",1,1,"1, 1024,1024,1",Counts,Pixel number,0,0.0,0,Internal,"0.,1.,0.,0.",2.100.30039.0,"5,00000","-74,1",1e-06,4,1.5,0,0,12.9E-06,Conventional,2.5x,4335,-999,False,0,0,0,0,0,500.0,0.0,0,0.0,0,0,0.0,0.0,0.0,2.802597e-45,0,1,1,-1,0,0.0,0,0,0,0,0,,,,,2019-01-31T06:44:53,2019-01-31T06:44:53.820,0,,,0,,0,0,,,"5,00000",2.5x,2019-01-31T06:44:53.820,14:28:50,-60:32:25,2000.0,1.389,2458514.78115,12:23:20,-2:06:52,HD126593_STD_L2_F2,0.60m(BC),0.34,Ikon 19002,HD126593,Julio Cesar Neves Campagnolo,6.3,L2,2,F2,2,2247,"756,0","18,8",51
True,16,2,1024,1024,True,32768,1,,Single,Image,"1, 1024,1024, 1",1,1,"1, 1024,1024,1",Counts,Pixel number,0,0.0,0,Internal,"0.,1.,0.,0.",2.100.30039.0,"5,00000","-74,1",1e-06,4,1.5,0,0,12.9E-06,Conventional,2.5x,4335,-999,False,0,0,0,0,0,500.0,0.0,0,0.0,0,0,0.0,0.0,0.0,2.802597e-45,0,1,1,-1,0,0.0,0,0,0,0,0,,,,,2019-01-31T06:45:06,2019-01-31T06:45:06.140,0,,,0,,0,0,,,"5,00000",2.5x,2019-01-31T06:45:06.140,14:28:50,-60:32:25,2000.0,1.389,2458514.78129,12:23:31,-2:06:40,HD126593_STD_L3_F2,0.60m(BC),0.34,Ikon 19002,HD126593,Julio Cesar Neves Campagnolo,6.3,L3,3,F2,2,2247,"756,0","18,8",51
True,16,2,1024,1024,True,32768,1,,Single,Image,"1, 1024,1024, 1",1,1,"1, 1024,1024,1",Counts,Pixel number,0,0.0,0,Internal,"0.,1.,0.,0.",2.100.30039.0,"5,00000","-74,1",1e-06,4,1.5,0,0,12.9E-06,Conventional,2.5x,4335,-999,Fa

## SQL implementation

In [ ]:
class FitsFileGroup_Table():
    """Easy handle groups of fits files."""

    def __init__(self, location=None, files=None, ext=0,
                 compression=False, database=':memory:', **kwargs):
        self._ext = ext
        self._extensions = kwargs.get('fits_ext',
                                      _fits_extensions_with_compress
                                      if compression else _fits_extensions)

        self._include = kwargs.get('glob_include')
        self._exclude = kwargs.get('glob_exclude')
        self._keywords = kwargs.get('keywords')
        self._db = Database(database, table='headers')

        if location is None and files is None:
            raise ValueError("You must specify a 'location'"
                             "or a list of 'files'")
        if files is None and location is not None:
            files = list_fits_files(location, self._extensions,
                                    self._include, self._exclude)

        self._files = files
        self._location = location
        
        self._read_db()

    def __len__(self):
        return len(self.files)

    @property
    def files(self):
        return self._files.copy()

    @property
    def location(self):
        return self._location

    @property
    def keywords(self):
        return self._keywords

    @property
    def summary(self):
        return Table(names=self._db.colnames(),
                     rows=self._db.select())

    def __copy__(self, files=None, db=None):
        nfg = FitsFileGroup.__new__(FitsFileGroup)
        for k, v in self.__dict__.items():
            if k == '_db':
                nfg._db = db or self._db
            elif k == '_files':
                nfg._files = files if files is not None else self._files
            else:
                nfg.__dict__[k] = v
        return nfg
    
    def _read_db(self):
        """Read the database."""

    def filtered(self, keywords=None):
        """Create a new FileGroup with only filtered files."""
        rows = self._db.select(where=keywords)
        raise NotImplementedError

    def values(self, keyword, unique=False):
        """Return the values of a keyword in the summary.

        If unique, only unique values returned.
        """
        raise NotImplementedError

    def add_column(self, name, values, mask=None):
        """Add a new column to the summary."""
        raise NotImplementedError

    def _intern_yelder(self, files=None, ext=None, ret_type=None,
                       **kwargs):
        """Iter over files."""
        ext = ext if ext is not None else self._ext
        files = files if files is not None else self._files
        for i in files:
            if ret_type == 'header':
                yield fits.open(i, **kwargs)[ext].header
            if ret_type == 'data':
                yield fits.open(i, **kwargs)[ext].data
            if ret_type == 'hdu':
                yield fits.open(i, **kwargs)[ext]
            if ret_type == 'framedata':
                yield check_framedata(i, hdu=ext, **kwargs)

    def hdus(self, ext=None, **kwargs):
        """Read the files and iterate over their HDUs."""
        return self._intern_yelder(ext=ext, ret_type='hdu', **kwargs)

    def headers(self, ext=None, **kwargs):
        """Read the files and iterate over their headers."""
        return self._intern_yelder(ext=ext, ret_type='header', **kwargs)

    def data(self, ext=None, **kwargs):
        """Read the files and iterate over their data."""
        return self._intern_yelder(ext=ext, ret_type='data', **kwargs)

    def framedata(self, ext=None, **kwargs):
        """Read the files and iterate over their data."""
        return self._intern_yelder(ext=ext, ret_type='framedata', **kwargs)